In [1]:
# !pip install openai

In [2]:
# !pip install --upgrade openai

In [1]:
import os
import openai
import time
import pandas as pd

In [1]:
openai.organization = ""#"X"
openai.api_key = "X"
openai.Model.list()

NameError: name 'openai' is not defined

In [3]:
template = lambda q, a: f"""<P {q} /P>. <R {a} /R>. <E"""
q = "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras."
a = "estamal por caben 20 dados"
prompt = template(q, a)
prompt

'<P Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras. /P>. <R estamal por caben 20 dados /R>. <E'

In [6]:
shots = [
    (template(
        "Agustín desea guardar unos dados de 1 cm cúbico en una caja que mide 5 cm de largo, 2 cm de ancho y 2 cm de alto. Su hermano dice que caben en la caja 15 dados.¿Es correcto lo que señala el hermano de Agustín? Explica con tus palabras.", 
        "esta mal  por caben 20 dados"), 
    "Coherente /E>"),
    (template(
        "María y su esposo ayer cocinaron una tortilla, la dividieron en 6 partes iguales. María comió 2/6 y su esposo 3/6. ¿Qué fracción de la tortilla quedo?", 
        "quedo un pedaso de tortilla y la ultima quedo a y la tortilla"), 
    "Coherente /E>"),
    (template(
        "Claudio tiene 12 lápices para repartir en cantidades iguales en 3 estuches, para hacerlo, su amiga Paula le dice que deberá poner 6 lápices en cada estuche y así no le sobrará ninguno. ¿Está en lo correcto la amiga de Claudio? (Explica con tus palabras la respuesta)", 
        "paula no esta en lo correcto porque me pide en mas estuches"), 
    "Coherente /E>"),
    (template(
        "Francisco y Camila tienen una colección de figuras de acción de su serie favorita, Francisco tiene 4/8 de la colección y Camila tiene 2/8, si del total de figuras que Francisco y Camila tienen pierden 5/8. ¿Cuántas figuras de la colección le quedan? (Redacta adecuadamente tu respuesta)", 
        "ESTA BUENA"), 
    "Incoherente /E>"),
]
def chatgpt(message, text, temp=0.7, max_t=5):
    try: 
        output = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                  {"role": "system", "content": message},
                  {"role": "user", "content": shots[0][0]},
                  {"role": "assistant", "content": shots[0][1]},
                  {"role": "user", "content": shots[1][0]},
                  {"role": "assistant", "content": shots[1][1]},
                  {"role": "user", "content": shots[2][0]},
                  {"role": "assistant", "content": shots[2][1]},
                  {"role": "user", "content": shots[3][0]},
                  {"role": "assistant", "content": shots[3][1]},
                  {"role": "user", "content": text},
            ],
            temperature=temp,
            max_tokens=max_t,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
    except:
        print("Thinking...")
        time.sleep(1)
        return chatgpt(message, text, temp=temp, max_t=max_t)
    return output

In [7]:
%%time
system = "Usted es un clasificador de incoherencia en las respuestas a preguntas abiertas de estudiantes de matemáticas de cuarto grado. Le daré una pregunta y respuestas diferentes, y luego tendrá que decir si la respuesta es coherente o incoherente a la pregunta. Marcaremos las preguntas entre <P /P> y las respuestas entre <R /R>. Entregará la etiqueta incoherente o coherente entre <E /E>."
chatgpt(system, prompt, temp=0.7, max_t=5)["choices"][0]["message"]["content"]

Wall time: 511 ms


'Incoherente /'

In [8]:
test_set = pd.read_excel("data/test_task_C1.xlsx", index_col=0)
test_set

,Q_id,Q,A,label
id,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1
...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1


In [9]:
q, a, label = test_set.sample(1).iloc[0]["Q A label".split()]
q, a, label

('Con los dígitos 240 y 6 crea un problema de multiplicación , con pregunta y respuesta',
 'SI',
 1)

In [10]:
%%time
responses = []
for i, ix in enumerate(test_set.index):
    q, a, label = test_set.loc[ix]["Q A label".split()]
    a = a.replace("\n", " ")
    a = " ".join(a.split(" ")).strip()
    o = chatgpt(system, template(q, a), temp=0.7, max_t=5)["choices"][0]["message"]["content"]
    o = o.replace("\n", " ").strip()
    responses.append([ix, o])
    print(100*(i+1)/test_set.shape[0], ix, o, label)

0.14771048744460857 85815 Incoherente / 0
0.29542097488921715 64812 Coherente /E 0
0.4431314623338257 84148 Incoherente / 0
0.5908419497784343 88565 Incoherente / 1
0.7385524372230429 51688 Incoherente / 1
0.8862629246676514 73857 Incoherente / 1
1.03397341211226 68843 Incoherente / 0
1.1816838995568686 73747 Incoherente / 0
1.329394387001477 92561 Incoherente / 1
1.4771048744460857 90379 Incoherente / 1
1.6248153618906942 84122 Coherente /E 0
1.7725258493353029 75775 Incoherente / 0
1.9202363367799113 56351 Incoherente / 0
2.06794682422452 85614 Coherente /E 0
2.2156573116691285 83090 Coherente /E 0
2.363367799113737 75196 Incoherente / 0
2.511078286558346 65885 Coherente /E 0
2.658788774002954 88549 Coherente /E 0
2.806499261447563 74418 Coherente /E 0
2.9542097488921715 60918 Incoherente / 0
3.1019202363367797 67940 Coherente /E 0
3.2496307237813884 61520 Incoherente / 0
3.397341211225997 51431 Incoherente / 0
3.5450516986706058 85988 Incoherente / 0
3.692762186115214 92045 Incohere

In [16]:
test_c1_pred = test_set.copy()
test_c1_pred["chatgpt"] = [x[1] for x in responses]
test_c1_pred

,Q_id,Q,A,label,chatgpt
id,,,,,
85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,Incoherente /
64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,Coherente /E
84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,Incoherente /
88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,Incoherente /
51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,Incoherente /
...,...,...,...,...,...
54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,Coherente /E
87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,Incoherente /
73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,Incoherente /


In [12]:
test_c1_pred.to_excel("chatgpt_responses_test_c1_pred_s4.xlsx")

In [13]:
from sklearn.metrics import classification_report

In [14]:
print(classification_report(test_c1_pred["label"], test_c1_pred["chatgpt"].apply(lambda x: int("incoherente" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.9282    0.3105    0.4654       541
           1     0.2480    0.9044    0.3892       136

    accuracy                         0.4298       677
   macro avg     0.5881    0.6075    0.4273       677
weighted avg     0.7915    0.4298    0.4501       677



In [15]:
root = "D:\\github\\coherence-prediction\\"
test_label_Q = pd.read_excel(root+"gpt_label_v7_comp.xlsx").set_index("id")
test_c1_pred["label_Q"] = test_label_Q.loc[test_c1_pred.index]["tipo_preg"]
print(classification_report(test_c1_pred[test_c1_pred["label_Q"] == 3]["label"], test_c1_pred[test_c1_pred["label_Q"] == 3]["chatgpt"].apply(lambda x: int("incoherente" in x.lower())), digits=4))

              precision    recall  f1-score   support

           0     0.9420    0.2481    0.3927       262
           1     0.1581    0.9024    0.2691        41

    accuracy                         0.3366       303
   macro avg     0.5501    0.5753    0.3309       303
weighted avg     0.8360    0.3366    0.3760       303

